In [1]:
import findspark

findspark.init('C:\\spark\\spark-2.3.1-bin-hadoop2.7')

In [2]:
import sys
assert sys.version_info >= (3, 5) # make sure we have Python 3.5+
import operator

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TopCategories").getOrCreate()
assert spark.version >= '2.3' # make sure we have Spark 2.3+
from pyspark import SparkContext
spark.sparkContext.setLogLevel('WARN')
sc = SparkContext
import re, operator, string
import json

In [5]:
def wwo(line):
    if line[0] is not None:
        x = line[0].split(",")
        for y in x:
            if len(y)>1:
                y = y.strip()
                yield(y,1)

In [24]:
business_table = spark.read.parquet('YELP_PAR/business')
review_table = spark.read.parquet('YELP_PAR/review')
all_categories = business_table.select("categories")
ccounts = all_categories.rdd.repartition(8).flatMap(wwo)
top10 = ccounts.reduceByKey(operator.add).sortBy(lambda x: x[1],ascending = False)
df = spark.createDataFrame(top10, ['category', 'count'])
df.limit(10).toPandas().to_csv("TopCategories.csv",index=False)